In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyvista as pv
import numpy as np
import scipy.linalg as la

np.set_printoptions(formatter={"float": "{:10.2f}".format})

mesh_path = "../tmp/INRIA_MUSIS.stl"
mesh = pv.read(mesh_path)

In [4]:
from src.dmcpworkflow.capture_depth import generate_depth_map

imIm, imK, imP = generate_depth_map(mesh_path)

pos = la.null_space(imP) / la.null_space(imP)[-1]
print(f"position \n{pos}")


position 
[[   6722.14]
 [  12345.64]
 [  10572.14]
 [      1.00]]


In [5]:
from src.dmcpworkflow.annotate_points import annotate

%matplotlib
cps = annotate(imIm,imIm)
%matplotlib inline
cps_annotated = cps.copy()
cps[:,2:] = cps[:,:2]

Using matplotlib backend: <object object at 0x7f1142f916b0>


from inria_musis_dmcp_transform we know that the pinhole matrix is not necessarily correct

thus we calibrate a new, correct one

In [10]:
from src.pycv.perspective import solve_PnP
from src.pycv.dmcp import dm_to_world


world_pts = dm_to_world(imIm, imK, imP, cps[:,:2])
pose_pnp = solve_PnP(world_pts, cps[:,:2], imK)
pose_pnp


array([[   1196.27,       0.00,     512.00],
       [      0.00,     897.20,     384.00],
       [      0.00,       0.00,       1.00]])

In [9]:
_mPts = pv.PolyData(world_pts)

pl = pv.Plotter()
pl.add_mesh(mesh)
pl.add_mesh(_mPts,color="red")
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)